In [93]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [94]:
print(f"inputs shape  : {inputs.shape}")

inputs shape  : torch.Size([6, 3])


In [95]:
query = inputs[1]
print(f"query : {query}")
attn_scores_2 = torch.empty(inputs.shape[0])
for i , x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i , query)
attn_scores_2

query : tensor([0.5500, 0.8700, 0.6600])


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [96]:
## normalize attn_scores_2
attn_scores_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(f"attn scores 2 tmp : {attn_scores_2_tmp}")

attn scores 2 tmp : tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [97]:
def softmax_naive(x):
  return torch.exp(x) / torch.exp(x).sum(dim = 0)
print(f"exp : {torch.exp(attn_scores_2)}")
attn_scores_2_naive = softmax_naive(attn_scores_2)
print(f"Attention weights : { attn_scores_2_naive }")
print(f"Sum : { attn_scores_2_naive.sum() }")

exp : tensor([2.5971, 4.4593, 4.3728, 2.3243, 2.0279, 2.9639])
Attention weights : tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum : 1.0


In [98]:
# 请注意，在处理大或小的输入值时，这种简单的软max实现（softmave）可能会遇到数值不稳定问题，如溢出和下流。
# 因此，在实践中，建议使用softmax的PyTorch实现，该实现已经对性能进行了广泛的优化：
attn_weights_2 = torch.softmax(attn_scores_2 , dim = 0)
print("Attention weights : " , attn_weights_2)
print("Sum : " , attn_weights_2.sum())

Attention weights :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


In [99]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i , x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i] * x_i 
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [100]:
## 3.3.2 Computing attention weights for all input tokens 
attn_scores = inputs @ inputs.T 
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [101]:
attn_weights = torch.softmax(attn_scores , dim = -1) 
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [102]:
all_context_vecs = attn_weights @ inputs 
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## 3.4 implementing self-attention with trainable weights

In [103]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
d_in

3

In [104]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in ,d_out) , requires_grad = False )
W_key = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad = False)

W_query , W_key , W_value

(Parameter containing:
 tensor([[0.2961, 0.5166],
         [0.2517, 0.6886],
         [0.0740, 0.8665]]),
 Parameter containing:
 tensor([[0.1366, 0.1025],
         [0.1841, 0.7264],
         [0.3153, 0.6871]]),
 Parameter containing:
 tensor([[0.0756, 0.1966],
         [0.3164, 0.4017],
         [0.1186, 0.8274]]))

In [105]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key 
value_2 = x_2 @W_value
query_2 , key_2 , value_2

(tensor([0.4306, 1.4551]), tensor([0.4433, 1.1419]), tensor([0.3951, 1.0037]))

In [106]:
keys = inputs @ W_key 
values = inputs @ W_value 
print(f"keys.shape {keys.shape} ")
print(f"values.shape {values.shape} ")

keys.shape torch.Size([6, 2]) 
values.shape torch.Size([6, 2]) 


In [107]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
attn_score_22


tensor(1.8524)

In [108]:
attn_scores_2 = query_2 @ keys.T 
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

In [109]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5 , dim= - 1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [110]:
context_vec_2 = attn_weights_2 @ values 
context_vec_2

tensor([0.3061, 0.8210])

## 3.4.2 Implementing a compact self-attention Python class


In [111]:
import torch.nn as nn 
class SelfAttention_v1(nn.Module):
  def __init__(self , d_in , d_out):
    super().__init__() 
    self.W_query = nn.Parameter(torch.rand(d_in , d_out))
    self.W_key = nn.Parameter(torch.rand(d_in , d_out))
    self.W_value = nn.Parameter(torch.rand(d_in , d_out))
  def forward(self, x):
    keys = x @ self.W_key
    queries = x @ self.W_query 
    values = x @ self.W_value
    attn_scores  = queries @ keys.T 
    attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5) , dim = -1)
    context_vec = attn_weights @ values 
    return context_vec
    

In [112]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [113]:
class SelfAttention_v2(nn.Module):
  def __init__(self , d_in , d_out , qkv_bias = False):
    super().__init__()
    self.W_query = nn.Linear(d_in, d_out , bias=qkv_bias)
    self.W_key = nn.Linear(d_in , d_out ,bias=qkv_bias)
    self.W_value = nn.Linear(d_in , d_out , bias=qkv_bias)
  
  def forward(self , x):
    keys = self.W_key(x)
    queries = self.W_query(x) 
    values = self.W_value(x) 
    attn_scores = queries @ keys.T 
    attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5 , dim=-1)
    context_vec = attn_weights @ values 
    return context_vec 
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in , d_out)
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

In [114]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [115]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length , context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [116]:
masked_simple = attn_weights * mask_simple 
masked_simple

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)

In [117]:
# normalize mask 
row_sums = masked_simple.sum(dim=-1, keepdim = True) 
masked_simple_norm = masked_simple/row_sums 
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [118]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [119]:
masked = attn_scores.masked_fill(mask.bool() , -torch.inf)
masked

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)

In [120]:
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5 , dim = 1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

## 3.5.2 Masking additional attention weights with dropout

In [121]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
dropout(example)

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])

## 3.5.3 Implementing a compact causal self-attention class


In [122]:
class CausalAttention(nn.Module):
  def __init__(self , d_in ,d_out , context_length , dropout , qkv_bias = False):
    super().__init__()
    self.d_out = d_out
    self.W_query = nn.Linear(d_in , d_out , bias = qkv_bias)
    self.W_key = nn.Linear(d_in , d_out , bias=qkv_bias)
    self.W_value = nn.Linear(d_in , d_out , bias = qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New
    
  def forward(self , x) : 
    b,num_tokens,d_in = x.shape
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)
    attn_scores = queries @ keys.transpose(1,2)
    attn_scores.masked_fill_(self.mask.bool()[:num_tokens , :num_tokens] , -torch.inf)
    attn_weights = torch.softmax(attn_scores/keys.shape[-1] ** 0.5 , dim = -1)
    attn_weights = self.dropout(attn_weights)
    context_vec = attn_weights @ values 
    return context_vec

In [123]:
batch = torch.stack((inputs, inputs) , dim = 0) 
batch.shape

torch.Size([2, 6, 3])

In [125]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in , d_out , context_length , 0.0)
context_vecs = ca(batch)
context_vecs

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)

In [126]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self , d_in , d_out , context_length , dropout , num_heads , qkv_bias = False):
    super().__init__() 
    self.heads = nn.ModuleList([CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)])
  def forward(self , x):
    return torch.cat([head(x) for head in self.heads] , dim = -1)
torch.manual_seed(123)
context_length = batch.shape[1]
d_in ,d_out = 3, 2 
mha = MultiHeadAttentionWrapper(d_in , d_out , context_length , 0.0 , num_heads = 2)
context_vecs = mha(batch)

context_vecs , context_vecs.shape

(tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
          [-0.5874,  0.0058,  0.5891,  0.3257],
          [-0.6300, -0.0632,  0.6202,  0.3860],
          [-0.5675, -0.0843,  0.5478,  0.3589],
          [-0.5526, -0.0981,  0.5321,  0.3428],
          [-0.5299, -0.1081,  0.5077,  0.3493]],
 
         [[-0.4519,  0.2216,  0.4772,  0.1063],
          [-0.5874,  0.0058,  0.5891,  0.3257],
          [-0.6300, -0.0632,  0.6202,  0.3860],
          [-0.5675, -0.0843,  0.5478,  0.3589],
          [-0.5526, -0.0981,  0.5321,  0.3428],
          [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>),
 torch.Size([2, 6, 4]))

## 3.6.2 Implementing Multi-head attention with weight splits

In [141]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in ,d_out , context_length , dropout , num_heads , qkv_bias = False):
    super().__init__()
    assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
    self.d_out = d_out 
    self.num_heads = num_heads 
    self.head_dim = d_out // num_heads 
    self.W_query = nn.Linear(d_in , d_out , bias = qkv_bias)
    self.W_key = nn.Linear(d_in , d_out , bias = qkv_bias)
    self.W_value = nn.Linear(d_in , d_out , bias = qkv_bias)
    self.out_proj = nn.Linear(d_out , d_out , bias = qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("mask" , torch.triu(torch.ones(context_length , context_length) , diagonal= 1))
  
  def forward(self , x):
    b, num_tokens , d_in = x.shape
    keys = self.W_key(x) 
    queries = self.W_query(x)
    values = self.W_value(x)
    print(f"a keys.shape : {keys.shape}")
    # We implicitly split the matrix by adding a `num_heads` dimension
    # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
    print(f"b keys.shape : {keys.shape}")
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)
    queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
    
    keys = keys.transpose(1,2)
    print(f"c keys.shape : {keys.shape}")
    queries = queries.transpose(1,2)
    values = values.transpose(1,2)
    
    attn_scores = queries @ keys.transpose(2,3)
    print(f"d keys transpose 2, 3 : {keys.transpose(2,3).shape}")
    print(f"attn_scores shape : {attn_scores.shape}")
    mask_bool = self.mask.bool()[:num_tokens,:num_tokens]
    attn_scores.masked_fill_(mask_bool , -torch.inf)
    attn_weights = torch.softmax(attn_scores / keys.shape[-1] * 0.5 , dim = -1)
    context_vec = (attn_weights @ values).transpose(1,2)
    context_vec = context_vec.contiguous().view(
      b, num_tokens, self.d_out
    )
    context_vec = self.out_proj(context_vec)
    return context_vec
    
    
torch.manual_seed(123)
batch_size , context_length , d_in = batch.shape 
print(f"""  batch_size : {batch_size}
          context_length : {context_length}
          d_in : {d_in}
      """)
d_out = 2 
mha = MultiHeadAttention(d_in , d_out , context_length , 0.0 , num_heads = 2)
context_vecs = mha(batch)
context_vecs , context_vecs.shape

  batch_size : 2
          context_length : 6
          d_in : 3
      
a keys.shape : torch.Size([2, 6, 8])
b keys.shape : torch.Size([2, 6, 2, 4])
c keys.shape : torch.Size([2, 2, 6, 4])
d keys transpose 2, 3 : torch.Size([2, 2, 4, 6])
attn_scores shape : torch.Size([2, 2, 6, 6])


(tensor([[[-0.1104,  0.4233,  0.2161, -0.0538, -0.4375,  0.2721, -0.2500,
            0.1410],
          [-0.0622,  0.4386,  0.3206,  0.0134, -0.4027,  0.2909, -0.2016,
            0.1826],
          [-0.0461,  0.4419,  0.3550,  0.0376, -0.3871,  0.2963, -0.1852,
            0.1971],
          [-0.0346,  0.3903,  0.3268,  0.0377, -0.3397,  0.2631, -0.1570,
            0.1769],
          [-0.0206,  0.3577,  0.3125,  0.0651, -0.2797,  0.2419, -0.1338,
            0.1775],
          [-0.0211,  0.3487,  0.3074,  0.0511, -0.2878,  0.2363, -0.1302,
            0.1669]],
 
         [[-0.1104,  0.4233,  0.2161, -0.0538, -0.4375,  0.2721, -0.2500,
            0.1410],
          [-0.0622,  0.4386,  0.3206,  0.0134, -0.4027,  0.2909, -0.2016,
            0.1826],
          [-0.0461,  0.4419,  0.3550,  0.0376, -0.3871,  0.2963, -0.1852,
            0.1971],
          [-0.0346,  0.3903,  0.3268,  0.0377, -0.3397,  0.2631, -0.1570,
            0.1769],
          [-0.0206,  0.3577,  0.3125,  0.0651, 